In [1]:
# Import the os package, used to set env variables and check paths
import os
# import the dotenv package for client keys in .env file
from dotenv import load_dotenv
# for random UUID keys
# https://docs.python.org/3/library/uuid.html
import uuid
# Import square 
from square.client import Client
# for pretty print
import pprint

In [2]:
# Get the current working directory
cwd = os.getcwd()

# Construct the .env file path
env_path = os.path.join(cwd, '.env')

# Load the .env file
load_dotenv(dotenv_path=env_path)

True

In [3]:
%%bash
#echo "SQUARE_ACCESS_TOKEN: ${SQUARE_ACCESS_TOKEN}"
#echo "SQUARE_APP_ID: ${SQUARE_APP_ID}"
#git clone https://github.com/square/square-python-sdk.git
#cd square-python-sdk
#pip install .


@hidden_cell

In [6]:
# init the square API
# using generic os environment this fails.
# However, when using the dotenv package it works
sq_client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='production'  
)


# Create a new item and associate with a new image

## step 1 create a catalog item and add to library

Everytime this is called, it creates a new one in the library.

Use `delete_catalog_object()` to delete them based upon obj_id

In [107]:
# create a key
uniq_uuid = uuid.uuid4()
#print("uniq_uuid: ", uniq_uuid)

# Specify the info we want to vary
#name="Nick Fury Doll"
#abbreviation = "NFDOLL"
#description = "something we want vertex ai to generate"
#variation_name = "6 inch poseable figure"
#amount = 350


# Specify the info we want to vary
name="Dr Doom Doll"
abbreviation = "DRDOLL"
description = "something we want vertex ai to generate"
variation_name = "6 inch poseable figure"
amount = 350


result = sq_client.catalog.upsert_catalog_object(
  body = {
    "idempotency_key": str(uniq_uuid),
    "object": {
      "type": "ITEM",
      "id": "#thisvaries1",
      "item_data": {
        "name": name,
        "description": description,
        "abbreviation": abbreviation,
        "variations": [
          {
            "type": "ITEM_VARIATION",
            "id": "#thisvaries2",
            "item_variation_data": {
              "item_id": "#thisvaries1",
              "name": variation_name,
              "pricing_type": "FIXED_PRICING",
              "price_money": {
                "amount": amount,
                "currency": "USD"
              }
            }
          },
        ]
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'catalog_object': {'type': 'ITEM', 'id': 'RVFBLY2M5FPCAFVUAT2CHGG4', 'updated_at': '2023-09-24T19:17:15.072Z', 'created_at': '2023-09-24T19:17:15.072Z', 'version': 1695583035072, 'is_deleted': False, 'present_at_all_locations': True, 'item_data': {'name': 'Dr Doom Doll', 'description': 'something we want vertex ai to generate', 'abbreviation': 'DRDOLL', 'is_taxable': True, 'variations': [{'type': 'ITEM_VARIATION', 'id': 'L7YGGF6SRMM6VY6V3QKSYMWQ', 'updated_at': '2023-09-24T19:17:15.072Z', 'created_at': '2023-09-24T19:17:15.072Z', 'version': 1695583035072, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': 'RVFBLY2M5FPCAFVUAT2CHGG4', 'name': '6 inch poseable figure', 'ordinal': 0, 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 350, 'currency': 'USD'}, 'sellable': True, 'stockable': True}}], 'product_type': 'REGULAR', 'description_html': '<p>something we want vertex ai to generate</p>', 'description_plaintext': 'something we want vertex

In [108]:
# print and save the object id
pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(result.body)

# Save the id
cat_obj_item_id = result.body['catalog_object']['id']
print(cat_obj_item_id)

RVFBLY2M5FPCAFVUAT2CHGG4


## Step 2 upload an image an associate with the catalog item


In [109]:
# create a key
uniq_uuid = uuid.uuid4()
#print("uniq_uuid: ", uniq_uuid)


# NICK Fury
#file_to_upload_path = "./sample_imgs/nickfury.jpeg" # Modify this to point to your desired file.

# DR DOOM
file_to_upload_path = "./sample_imgs/drdoom.jpeg" # Modify this to point to your desired file.


f_stream = open(file_to_upload_path, "rb")



result = sq_client.catalog.create_catalog_image(
  request = {
    "idempotency_key": str(uniq_uuid),
     "object_id": cat_obj_item_id,
    "image": {
      "type": "IMAGE",
      "id": "#image_id",
      "image_data": {
        "name": "NickFury Doll Image",
        "caption": "image caption nick fury"
      }
    }
  },
  image_file = f_stream
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'image': {'type': 'IMAGE', 'id': 'OW6GLIMXRCZZUXTZSEEUMSOB', 'updated_at': '2023-09-24T13:26:42.431Z', 'created_at': '2023-09-24T13:26:42.431Z', 'version': 1695562002431, 'is_deleted': False, 'present_at_all_locations': True, 'image_data': {'name': 'Image name', 'url': 'https://items-images-production.s3.us-west-2.amazonaws.com/files/0a980973e5af640e65bc451ff2f4c565a2819d9e/original.jpeg', 'caption': 'Image caption'}}}


# verify the catalog has the item we added

## Query all items and look through results manually

In [110]:
result = sq_client.catalog.list_catalog()

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)



{'objects': [{'type': 'CUSTOM_ATTRIBUTE_DEFINITION', 'id': 'TN6I6F7QXTEKDRT4M5F6JA7J', 'updated_at': '2023-09-17T16:01:13.582Z', 'created_at': '2023-09-17T16:01:13.582Z', 'version': 1694966473582, 'is_deleted': False, 'present_at_all_locations': True, 'custom_attribute_definition_data': {'type': 'BOOLEAN', 'name': 'Is Alcoholic', 'description': 'Enabling this toggle on an item indicates that it contains alcohol.', 'source_application': {'application_id': 'sq0idp-w46nJ_NCNDMSOywaCY0mwA', 'name': 'Square Online Store'}, 'allowed_object_types': ['ITEM'], 'seller_visibility': 'SELLER_VISIBILITY_HIDDEN', 'app_visibility': 'APP_VISIBILITY_HIDDEN', 'key': 'is_alcoholic'}}, {'type': 'CUSTOM_ATTRIBUTE_DEFINITION', 'id': 'KKJDEQ5ET5QY232B3XO4GRR2', 'updated_at': '2023-09-17T16:01:13.869Z', 'created_at': '2023-09-17T16:01:13.869Z', 'version': 1694966473869, 'is_deleted': False, 'present_at_all_locations': True, 'custom_attribute_definition_data': {'type': 'STRING', 'name': 'Ecom Storefront Classi

In [111]:
pp.pprint(result.body)

{   'objects': [   {   'created_at': '2023-09-17T16:01:13.582Z',
                       'custom_attribute_definition_data': {   'allowed_object_types': [   'ITEM'],
                                                               'app_visibility': 'APP_VISIBILITY_HIDDEN',
                                                               'description': 'Enabling '
                                                                              'this '
                                                                              'toggle '
                                                                              'on '
                                                                              'an '
                                                                              'item '
                                                                              'indicates '
                                                                              'that '
                                  


## Query API, finds an object based upon characteristics

If you know the object ID, Retrieve Catalog Obj API in next section

### Query for a specific item by attribute (name)

In [112]:
result = sq_client.catalog.search_catalog_objects(
  body = {
    "query": {
      "exact_query": {
        "attribute_name": "name",
        "attribute_value": "Nick Fury Doll"
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'objects': [{'type': 'ITEM', 'id': '6DFQU7YDLUEUYWEQNG3JADY7', 'updated_at': '2023-09-24T19:16:53.95Z', 'created_at': '2023-09-24T19:07:45.244Z', 'version': 1695583013950, 'is_deleted': False, 'present_at_all_locations': True, 'item_data': {'name': 'Nick Fury Doll', 'description': 'something we want vertex ai to generate', 'abbreviation': 'NFDOLL', 'is_taxable': True, 'variations': [{'type': 'ITEM_VARIATION', 'id': '7WKL7C5WGOIX2FR55Z5BADEE', 'updated_at': '2023-09-24T19:16:53.95Z', 'created_at': '2023-09-24T19:07:45.244Z', 'version': 1695583013950, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': '6DFQU7YDLUEUYWEQNG3JADY7', 'name': '6 inch poseable figure', 'ordinal': 0, 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 350, 'currency': 'USD'}, 'sellable': True, 'stockable': True, 'channels': ['CH_JUmVwhwpkvbQx4R8gulOlRgqxNIFwY9iO4pfYRlQuYC']}}], 'product_type': 'REGULAR', 'ecom_uri': 'https://rtp-gcp-usergroup.square.site/product/nic

In [113]:
pp.pprint(result.body)

{   'latest_time': '2023-09-24T19:17:19.334Z',
    'objects': [   {   'created_at': '2023-09-24T19:07:45.244Z',
                       'id': '6DFQU7YDLUEUYWEQNG3JADY7',
                       'is_deleted': False,
                       'item_data': {   'abbreviation': 'NFDOLL',
                                        'channels': [   'CH_JUmVwhwpkvbQx4R8gulOlRgqxNIFwY9iO4pfYRlQuYC'],
                                        'description': 'something we want '
                                                       'vertex ai to generate',
                                        'description_html': '<p>something we '
                                                            'want vertex ai to '
                                                            'generate</p>',
                                        'description_plaintext': 'something we '
                                                                 'want vertex '
                                                             

## Query for a specific item by attribute (abbreviation)

In [114]:
# I could not determine a way to query on the ID
# If you used attribute, you can specify the abbreviation but not the ID given after the ID>

result = sq_client.catalog.search_catalog_objects(
  body = {
    "object_types": [
      "ITEM"
    ],
    "query": {
      "exact_query": {
        "attribute_name": "abbreviation",
        "attribute_value": "NFDOLL"
      }
    }
  }
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'objects': [{'type': 'ITEM', 'id': '6DFQU7YDLUEUYWEQNG3JADY7', 'updated_at': '2023-09-24T19:16:53.95Z', 'created_at': '2023-09-24T19:07:45.244Z', 'version': 1695583013950, 'is_deleted': False, 'present_at_all_locations': True, 'item_data': {'name': 'Nick Fury Doll', 'description': 'something we want vertex ai to generate', 'abbreviation': 'NFDOLL', 'is_taxable': True, 'variations': [{'type': 'ITEM_VARIATION', 'id': '7WKL7C5WGOIX2FR55Z5BADEE', 'updated_at': '2023-09-24T19:16:53.95Z', 'created_at': '2023-09-24T19:07:45.244Z', 'version': 1695583013950, 'is_deleted': False, 'present_at_all_locations': True, 'item_variation_data': {'item_id': '6DFQU7YDLUEUYWEQNG3JADY7', 'name': '6 inch poseable figure', 'ordinal': 0, 'pricing_type': 'FIXED_PRICING', 'price_money': {'amount': 350, 'currency': 'USD'}, 'sellable': True, 'stockable': True, 'channels': ['CH_JUmVwhwpkvbQx4R8gulOlRgqxNIFwY9iO4pfYRlQuYC']}}], 'product_type': 'REGULAR', 'ecom_uri': 'https://rtp-gcp-usergroup.square.site/product/nic

In [115]:
pp.pprint(result.body)

{   'latest_time': '2023-09-24T19:17:19.334Z',
    'objects': [   {   'created_at': '2023-09-24T19:07:45.244Z',
                       'id': '6DFQU7YDLUEUYWEQNG3JADY7',
                       'is_deleted': False,
                       'item_data': {   'abbreviation': 'NFDOLL',
                                        'channels': [   'CH_JUmVwhwpkvbQx4R8gulOlRgqxNIFwY9iO4pfYRlQuYC'],
                                        'description': 'something we want '
                                                       'vertex ai to generate',
                                        'description_html': '<p>something we '
                                                            'want vertex ai to '
                                                            'generate</p>',
                                        'description_plaintext': 'something we '
                                                                 'want vertex '
                                                             

## To read info about a particular item use "Retrieve Catalog Object"

In [116]:
# IDS                             Name               ABBR             Description                                  NOTES
# 6DFQU7YDLUEUYWEQNG3JADY7        Nick Fury Doll     NFDOLL           something about Nick Fury from vertex ai
# ULVIYDNW5P6CMV5QTYFL2ABJ        Dr Doom Doll       DRDOLL           something we want vertext ai to generate


In [117]:
# nick fury
#object_id = '6DFQU7YDLUEUYWEQNG3JADY7'
# dr doom
object_id = 'ULVIYDNW5P6CMV5QTYFL2ABJ'
result = sq_client.catalog.retrieve_catalog_object(
  object_id = object_id
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

[{'category': 'INVALID_REQUEST_ERROR', 'code': 'NOT_FOUND', 'detail': 'Catalog object with ID `ULVIYDNW5P6CMV5QTYFL2ABJ` not found.'}]


In [118]:
pp.pprint(result.body)

{   'errors': [   {   'category': 'INVALID_REQUEST_ERROR',
                      'code': 'NOT_FOUND',
                      'detail': 'Catalog object with ID '
                                '`ULVIYDNW5P6CMV5QTYFL2ABJ` not found.'}]}


## Delete a catalog object item

`DONT RUN UNLESS YOU WANT TO RECREATE THE ITEM`

In [106]:
delete_id = "ULVIYDNW5P6CMV5QTYFL2ABJ"
result = sq_client.catalog.delete_catalog_object(
  object_id = delete_id
)

if result.is_success():
  print(result.body)
elif result.is_error():
  print(result.errors)

{'deleted_object_ids': ['7WB6SOMSLRRL3UDFZPGKGU4G', 'ULVIYDNW5P6CMV5QTYFL2ABJ'], 'deleted_at': '2023-09-24T19:17:03.484Z'}
